# Research Notebook: Working with Fabric Core APIs to handle deployment and lifecycle management of Fabric items

This notebook demonstrates how to use the Fabric Core APIs to handle deployment and lifecycle management of Fabric items. The Fabric Core APIs are a set of APIs that allow you to interact with the Fabric platform programmatically. The Fabric Core APIs provide a way to automate common tasks such as deploying items, updating items, and managing the lifecycle of items.

In [1]:
#r "nuget: Azure.Identity, 1.13.1"
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: Microsoft.Net.Http,  2.2.29"
#r "nuget: Microsoft.Fabric.Api, 1.0.0-beta"


Installed Packages Azure.Identity, 1.13.1 DotNetEnv, 2.5.0 Microsoft.Fabric.Api, 1.0.0-beta Microsoft.Net.Http, 2.2.29

In [2]:
using Microsoft.Identity.Client;
using System;
using System.Threading.Tasks;
using System.Net.Http;
using System.Net.Http.Headers;
using DotNetEnv;

## Loading Environment Variables

The following variables are stored in the `.env` file:
- `AZURE_TENANT_ID`: The Azure tenant ID.
- `FABRIC_API_CLIENT_ID`: The client ID of the app registration governing access to the Fabric API.
- `FABRIC_API_CLIENT_SECRET`: The client secret of the app registration governing access to the Fabric API.

These would be used to acquire an access token to authenticate requests to the Fabric API.

In [3]:
static string _configurationFile = @".env";
Env.Load(_configurationFile);

// client cred flow
string client_secret = Environment.GetEnvironmentVariable("FABRIC_API_CLIENT_SECRET");
string tenantId = Environment.GetEnvironmentVariable("AZURE_TENANT_ID");
string clientId = Environment.GetEnvironmentVariable("FABRIC_API_CLIENT_ID");


string[] scopes = new string[] { "https://api.fabric.microsoft.com/.default" };
string authority = $"https://login.microsoftonline.com/{tenantId}";
// check env variables
if (string.IsNullOrEmpty(client_secret) || string.IsNullOrEmpty(tenantId) || string.IsNullOrEmpty(clientId))
{
    throw new Exception("Please provide all the required environment variables");
}else{
    Console.WriteLine("All environment variables are set");
    Console.WriteLine($"tenantId: {tenantId} clientId: {clientId}");
}

All environment variables are set
tenantId: 0a8c004f-f29f-416b-82be-db6db348e5fa clientId: f052e5d7-abea-41ed-b29d-5f50cd49b726


## Access token acquisition & caching

Before all calls to the fabric api endpoints, we need to acquire an access token. The method `GetAccessTokenAsync` is used to verify the existence of a valid access token, if it is expired or does not exist, a new access token is acquired.

In [5]:
// Token cache variables
string accessToken = null;
DateTimeOffset accessTokenExpiresOn = DateTimeOffset.MinValue;
IConfidentialClientApplication app = ConfidentialClientApplicationBuilder.Create(clientId)
    .WithClientSecret(client_secret)
    .WithAuthority(authority)
    .Build();

async Task<string> GetAccessTokenAsync()
{
    // Check if token is still valid
    if (!string.IsNullOrEmpty(accessToken) && DateTimeOffset.UtcNow < accessTokenExpiresOn)
    {
        // Token is still valid
        Console.WriteLine("Token is still valid");
        return accessToken;
    }

    // Token is expired or not acquired yet
    try
    {
        AuthenticationResult result = await app.AcquireTokenForClient(scopes)
            .ExecuteAsync();

        // Update the token and its expiration time
        accessToken = result.AccessToken;
        
        accessTokenExpiresOn = result.ExpiresOn;

        return accessToken;
    }
    catch (MsalServiceException ex)
    {
        Console.WriteLine($"MsalServiceException when acquiring token: {ex.Message}");
        throw;
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Exception when acquiring token: {ex.Message}");
        throw;
    }
}

## Supporting Methods

The current design use a string representation of the api response, casting to specific objects would be done post request.
The following methods are used to interact with the Fabric Core APIs:
- `GetAsync`: This method is used to make a GET request to the Fabric API. It would appends the provided uri to the base url and returns the response as a string.
- `PostAsync`: this method is used to make a POST request to the Fabric API. It would appends the provided uri to the base url and returns the response as a string. The payload is provided as `HttpContent`.

In [6]:
HttpClient httpClient;
httpClient = new HttpClient
    {
        BaseAddress = new Uri("https://api.fabric.microsoft.com/v1/")
    };
    
async Task<string> GetAsync(string uri)
{
    string accessToken = await GetAccessTokenAsync();
    httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", accessToken);

    try
    {
        HttpResponseMessage response = await httpClient.GetAsync(uri);
        response.EnsureSuccessStatusCode(); // Throws if not 2xx

        string responseBody = await response.Content.ReadAsStringAsync();
        return responseBody;
    }
    catch (HttpRequestException ex)
    {
        Console.WriteLine($"HTTP GET error: {ex.Message}");
        throw;
    }
}

async Task<string> PostAsync(string uri, string jsonPayload)
{
    string accessToken = await GetAccessTokenAsync();
    httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", accessToken);

    try
    {
        var content = new StringContent(jsonPayload, Encoding.UTF8, "application/json");
        HttpResponseMessage response = await httpClient.PostAsync(uri, content);

        if (response.StatusCode == System.Net.HttpStatusCode.Accepted) // Check for 202
        {
            if (response.Headers.Location != null)
            {
                return response.Headers.Location.ToString(); // Return the Location header
            }
            throw new Exception("202 Accepted but no Location header returned.");
        }

        response.EnsureSuccessStatusCode(); // Throws if not 2xx
        return await response.Content.ReadAsStringAsync();
    }
    catch (HttpRequestException ex)
    {
        Console.WriteLine($"HTTP POST error: {ex.Message}");
        throw;
    }
}
async Task<string> PollStatusAsync(string locationUri)
{
    string accessToken = await GetAccessTokenAsync();
    httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", accessToken);

    while (true)
    {
        try
        {
            HttpResponseMessage response = await httpClient.GetAsync(locationUri);

            if (response.StatusCode == System.Net.HttpStatusCode.OK)
            {
                // Operation completed; return the final response
                return await response.Content.ReadAsStringAsync();
            }
            else if (response.StatusCode == System.Net.HttpStatusCode.Accepted)
            {
                // Operation still in progress; wait and retry
                Console.WriteLine("Operation in progress...");
                await Task.Delay(2000); // Wait for 2 seconds before retrying
                continue;
            }
            else
            {
                // Handle other status codes
                throw new Exception($"Unexpected status code: {response.StatusCode}");
            }
        }
        catch (HttpRequestException ex)
        {
            Console.WriteLine($"Polling error: {ex.Message}");
            throw;
        }
    }
}

async Task DeleteAsync(string uri)
{
    string accessToken = await GetAccessTokenAsync();
    httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", accessToken);

    try
    {
        HttpResponseMessage response = await httpClient.DeleteAsync(uri);

        if (response.IsSuccessStatusCode)
        {
            Console.WriteLine($"Successfully deleted: {uri}");
        }
        else
        {
            Console.WriteLine($"Failed to delete: {uri}");
            Console.WriteLine($"Status Code: {response.StatusCode}");
            Console.WriteLine($"Reason: {response.ReasonPhrase}");
        }
    }
    catch (HttpRequestException ex)
    {
        Console.WriteLine($"HTTP DELETE error: {ex.Message}");
        throw;
    }
}

async Task<string> PatchAsync(string uri, string jsonPayload)
{
    string accessToken = await GetAccessTokenAsync();
    httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", accessToken);

    try
    {
        var content = new StringContent(jsonPayload, Encoding.UTF8, "application/json");
        var request = new HttpRequestMessage(new HttpMethod("PATCH"), uri)
        {
            Content = content
        };

        HttpResponseMessage response = await httpClient.SendAsync(request);

        if (response.IsSuccessStatusCode)
        {
            Console.WriteLine($"Successfully patched: {uri}");
            return await response.Content.ReadAsStringAsync(); // Return response body if needed
        }
        else
        {
            Console.WriteLine($"Failed to patch: {uri}");
            Console.WriteLine($"Status Code: {response.StatusCode}");
            Console.WriteLine($"Reason: {response.ReasonPhrase}");
            string errorContent = await response.Content.ReadAsStringAsync();
            Console.WriteLine($"Error Content: {errorContent}");
            throw new Exception($"PATCH request failed with status code {response.StatusCode}");
        }
    }
    catch (HttpRequestException ex)
    {
        Console.WriteLine($"HTTP PATCH error: {ex.Message}");
        throw;
    }
}

## Examples

### Getting an Access Token

In [7]:
var token = GetAccessTokenAsync().Result;
Console.WriteLine(token);   

eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Inp4ZWcyV09OcFRrd041R21lWWN1VGR0QzZKMCIsImtpZCI6Inp4ZWcyV09OcFRrd041R21lWWN1VGR0QzZKMCJ9.eyJhdWQiOiJodHRwczovL2FwaS5mYWJyaWMubWljcm9zb2Z0LmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzBhOGMwMDRmLWYyOWYtNDE2Yi04MmJlLWRiNmRiMzQ4ZTVmYS8iLCJpYXQiOjE3MzIyMDE1ODgsIm5iZiI6MTczMjIwMTU4OCwiZXhwIjoxNzMyMjA1NDg4LCJhaW8iOiJrMkJnWUxCbVc2QTY1Y0g3ME1RTnUvaUxidlUxQUFBPSIsImFwcGlkIjoiZjA1MmU1ZDctYWJlYS00MWVkLWIyOWQtNWY1MGNkNDliNzI2IiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMGE4YzAwNGYtZjI5Zi00MTZiLTgyYmUtZGI2ZGIzNDhlNWZhLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiNzI0MjEwNjYtYTNjOC00ZjhhLWJmYzYtM2M1NjE4MDZjMjQ3IiwicmgiOiIxLkFXRUJUd0NNQ3BfeWEwR0N2dHR0czBqbC1na0FBQUFBQUFBQXdBQUFBQUFBQUFCaEFRQmhBUS4iLCJyb2xlcyI6WyJUZW5hbnQuUmVhZFdyaXRlLkFsbCIsIlRlbmFudC5SZWFkLkFsbCJdLCJzdWIiOiI3MjQyMTA2Ni1hM2M4LTRmOGEtYmZjNi0zYzU2MTgwNmMyNDciLCJ0aWQiOiIwYThjMDA0Zi1mMjlmLTQxNmItODJiZS1kYjZkYjM0OGU1ZmEiLCJ1dGkiOiJmak05MlNQN2cwaUp4YUdGU1FFUkFBIiwidmVyIjoiMS4wIiwieG1zX2l

### Getting all the workspaces

In [8]:

string workspacesResponse = await GetAsync("workspaces");
Console.WriteLine("Workspaces:");
Console.WriteLine(workspacesResponse);



Token is still valid
Workspaces:
{"value":[{"id":"4aca0508-e966-40a7-a2a2-84861b281c83","displayName":"feature6-11","description":"demo feature workspace","type":"Workspace","capacityId":"5eac7a35-45d2-4c15-b2f1-b1aabee4ae2e"},{"id":"3dc6d3ea-f6c5-41c6-b9f7-dbdadccfd76e","displayName":"testwithdolev","description":"","type":"Workspace","capacityId":"5eac7a35-45d2-4c15-b2f1-b1aabee4ae2e"}]}


### Getting items in a workspace

In [9]:
// use one of the ids from the workspaces response
string sample_ws_id = "4aca0508-e966-40a7-a2a2-84861b281c83";
string itemsResponse = await GetAsync($"workspaces/{sample_ws_id}/notebooks");
Console.WriteLine("Items:");
Console.WriteLine(itemsResponse);

Token is still valid
Items:
{"value":[{"id":"82098dd0-0b5c-42c0-9060-97600acd3063","type":"Notebook","displayName":"Notebook 2","description":"New notebook","workspaceId":"4aca0508-e966-40a7-a2a2-84861b281c83"},{"id":"31e8534b-d7a7-4213-a0d5-7a2630e11473","type":"Notebook","displayName":"Notebook X","description":"A notebook which was created via REST description","workspaceId":"4aca0508-e966-40a7-a2a2-84861b281c83"},{"id":"e5c54f32-4b1b-4289-9654-814309a0bb14","type":"Notebook","displayName":"Notebook 2XX","description":"A notebook which was created via REST together with O","workspaceId":"4aca0508-e966-40a7-a2a2-84861b281c83"},{"id":"6e072d2a-587c-49a6-ae3f-92386730dc7f","type":"Notebook","displayName":"Notebook 4X4","description":"A notebook which was created via REST description","workspaceId":"4aca0508-e966-40a7-a2a2-84861b281c83"},{"id":"ee37a9ae-d1d8-4ac8-b132-debb0279a1b0","type":"Notebook","displayName":"A Sample Notebook4 Updated","description":"A sample notebook4 description

## Deployment and Lifecycle Management

In this section, we would demonstrate how to deploy items to a workspace and manage the lifecycle of the items. (e.g. create, update, delete)

The [REST](https://learn.microsoft.com/en-us/rest/api/fabric/notebook/items) documentation provide further details.

### Create a new item : Notebook : w/o definition

Creation of an item (including a notebook) is creating an empty content asset and a meta-data asset named `.platform`. The meta-data asset contains the definition of the item. Here is how the payload would look like for a creation of an item without a definition.
One needs to provide the `workspaceId` in which the item would be created.

```json
POST https://api.fabric.microsoft.com/v1/workspaces/{{workspaceId}}/notebooks
Content-Type: application/json
Authorization: Bearer ACCESS_TOKEN

{
    "displayName": "Notebook Name",
    "description": "A notebook which was created via REST API"
}
```


In [10]:
using System.Text.Json.Serialization;
using System.Text.Json;

public class NotebookPayload
{
    [JsonPropertyName("displayName")]
    public string DisplayName { get; set; }

    [JsonPropertyName("description")]
    public string Description { get; set; }

    [JsonPropertyName("definition")]
    public NotebookDefinition Definition { get; set; }
}

public class NotebookDefinition
{
    // dont use the format
    // [JsonPropertyName("format")]
    // public string Format { get; set; }

    [JsonPropertyName("parts")]
    public List<NotebookPart> Parts { get; set; }
}

public class NotebookPart
{
    [JsonPropertyName("path")]
    public string Path { get; set; }

    [JsonPropertyName("payload")]
    public string Payload { get; set; }

    [JsonPropertyName("payloadType")]
    public string PayloadType { get; set; }
}

public class NotebookItem
{
    [JsonPropertyName("id")]
    public string Id { get; set; }

    [JsonPropertyName("type")]
    public string Type { get; set; }

    [JsonPropertyName("displayName")]
    public string DisplayName { get; set; }

    [JsonPropertyName("description")]
    public string Description { get; set; }

    [JsonPropertyName("workspaceId")]
    public string WorkspaceId { get; set; }
}

public class NotebookResponse
{
    [JsonPropertyName("value")]
    public List<NotebookItem> Value { get; set; }
}

In [11]:
string notebook_name = "A Sample Notebook5";
string notebook_desc = "A sample notebook5 description5";

var notebookPayload = new NotebookPayload
{
    DisplayName = notebook_name,
    Description = notebook_desc
};

var uri = $"workspaces/{sample_ws_id}/notebooks";
var jsonPayload = JsonSerializer.Serialize(notebookPayload);

// Step 1: Create the notebook
string location = await PostAsync(uri, jsonPayload);
Console.WriteLine($"Notebook creation initiated. Location: {location}");

// Step 2: Poll the status
if (!string.IsNullOrEmpty(location))
{
    string finalResponse = await PollStatusAsync(location);
    Console.WriteLine("Final Notebook Response:");
    Console.WriteLine(finalResponse);
}
else
{
    Console.WriteLine("No Location header received; unable to poll.");
}

Token is still valid
Notebook creation initiated. Location: https://wabi-us-east2-d-primary-redirect.analysis.windows.net/v1/operations/fb93f9ff-df5b-44d4-aba0-929bfbeba07c
Token is still valid
Final Notebook Response:
{"status":"Succeeded","createdTimeUtc":"2024-11-21T15:12:18.7573396","lastUpdatedTimeUtc":"2024-11-21T15:12:19.2729699","percentComplete":100,"error":null}


After a notebook is created, we can either delete it or update it. However the response does not include the id of the notebook. for that we need to call yet another api.

### List Notebooks

`GET https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks`

In [14]:
// using the same sample_ws_id

var uri = $"workspaces/{sample_ws_id}/notebooks";

string itemsResponse = await GetAsync(uri);
try
{
    // Deserialize the JSON response
    var notebooks = JsonSerializer.Deserialize<NotebookResponse>(itemsResponse);

    // Print in a readable format
    Console.WriteLine("Items:");
    foreach (var notebook in notebooks?.Value ?? new List<NotebookItem>())
    {
        Console.WriteLine($"- ID: {notebook.Id}");
        Console.WriteLine($"  Type: {notebook.Type}");
        Console.WriteLine($"  Display Name: {notebook.DisplayName}");
        Console.WriteLine($"  Description: {notebook.Description}");
        Console.WriteLine($"  Workspace ID: {notebook.WorkspaceId}");
        Console.WriteLine(); // Blank line for better readability
    }
}
catch (Exception ex)
{
    Console.WriteLine($"Error processing response: {ex.Message}");
}

Token is still valid
Items:
- ID: 82098dd0-0b5c-42c0-9060-97600acd3063
  Type: Notebook
  Display Name: Notebook 2
  Description: New notebook
  Workspace ID: 4aca0508-e966-40a7-a2a2-84861b281c83

- ID: 31e8534b-d7a7-4213-a0d5-7a2630e11473
  Type: Notebook
  Display Name: Notebook X
  Description: A notebook which was created via REST description
  Workspace ID: 4aca0508-e966-40a7-a2a2-84861b281c83

- ID: e5c54f32-4b1b-4289-9654-814309a0bb14
  Type: Notebook
  Display Name: Notebook 2XX
  Description: A notebook which was created via REST together with O
  Workspace ID: 4aca0508-e966-40a7-a2a2-84861b281c83

- ID: 6e072d2a-587c-49a6-ae3f-92386730dc7f
  Type: Notebook
  Display Name: Notebook 4X4
  Description: A notebook which was created via REST description
  Workspace ID: 4aca0508-e966-40a7-a2a2-84861b281c83

- ID: ee37a9ae-d1d8-4ac8-b132-debb0279a1b0
  Type: Notebook
  Display Name: A Sample Notebook4 Updated
  Description: A sample notebook4 description4 Updated
  Workspace ID: 4ac

### Delete a Notebook

In order to delete we need to have the notebook id and the workspace id to which it belongs:
`DELETE https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks/{notebookId}`

In [13]:
var notebook_id = "494bbb26-aa14-4ec6-8896-4929722a8b71";
var deleteUri = $"workspaces/{sample_ws_id}/notebooks/{notebook_id}";
await DeleteAsync(deleteUri);



Token is still valid
Successfully deleted: workspaces/4aca0508-e966-40a7-a2a2-84861b281c83/notebooks/494bbb26-aa14-4ec6-8896-4929722a8b71


### Update a Notebook

Lets first examine the update w/o the definition of the notebook:
`PATCH https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks/{notebookId}` is the api we need to call. it has similar payload as the creation w/o definition.

In [ ]:
var notebook_2_update = "ee37a9ae-d1d8-4ac8-b132-debb0279a1b0";
var notebook_name = "A Sample Notebook4 Updated";
var notebook_desc = "A sample notebook4 description4 Updated";

var notebookPayload = new NotebookPayload
{
    DisplayName = notebook_name,
    Description = notebook_desc
};

var uri = $"workspaces/{sample_ws_id}/notebooks/{notebook_2_update}";
var jsonPayload = JsonSerializer.Serialize(notebookPayload);

// Call the PatchAsync method
string patchResponse = await PatchAsync(uri, jsonPayload);
Console.WriteLine("Patch Response:");
Console.WriteLine(patchResponse);

### Create Item (Notebook) with Definition

The definition of the notebook is a json object which contains the cells (content) and the metadata of the notebook (.platform). The payload would look like this:

```json
POST https://api.fabric.microsoft.com/v1/workspaces/{{workspaceId}}/notebooks
Content-Type: application/json
Authorization Bearer ACCESS

{
    "displayName": "Notebook XXX",
    "description": "A notebook which was created via REST description",
    "definition": {
        "parts": [
            {
                "path": "notebook-content.py",
                "payload": "{{content_b64}}",
                "payloadType": "InlineBase64"
            },
            {
                "path": ".platform",
                "payload": "{{platform_b64}}",
                "payloadType": "InlineBase64"
            }
        ]
    }
}
```

In [15]:
// will continue to use the same sample_ws_id

var notebook_with_name = "Sample Notebook 5x5";
var notebook_with_desc = "Sample Notebook 5x5 description with definition";

var platform_part_b64 = "ewogICIkc2NoZW1hIjogImh0dHBzOi8vZGV2ZWxvcGVyLm1pY3Jvc29mdC5jb20vanNvbi1zY2hlbWFzL2ZhYnJpYy9naXRJbnRlZ3JhdGlvbi9wbGF0Zm9ybVByb3BlcnRpZXMvMi4wLjAvc2NoZW1hLmpzb24iLAogICJtZXRhZGF0YSI6IHsKICAgICJ0eXBlIjogIk5vdGVib29rIiwKICAgICJkaXNwbGF5TmFtZSI6ICJOb3RlYm9vayAyWFgiLAogICAgImRlc2NyaXB0aW9uIjogIk5ldyBub3RlYm9vayBmcmVzaCBmcm9tIFJFU1Qgd2l0aCBzZWNyZXQgc2F1Y2UiCiAgfSwKICAiY29uZmlnIjogewogICAgInZlcnNpb24iOiAiMi4wIiwKICAgICJsb2dpY2FsSWQiOiAiMDAwMDAwMDAtMDAwMC0wMDAwLTAwMDAtMDAwMDAwMDAwMDAwIgogIH0KfQ==";
var content_part_b64 = "IyBGYWJyaWMgbm90ZWJvb2sgc291cmNlCgojIE1FVEFEQVRBICoqKioqKioqKioqKioqKioqKioqCgojIE1FVEEgewojIE1FVEEgICAia2VybmVsX2luZm8iOiB7CiMgTUVUQSAgICAgIm5hbWUiOiAic3luYXBzZV9weXNwYXJrIgojIE1FVEEgICB9LAojIE1FVEEgICAiZGVwZW5kZW5jaWVzIjogewojIE1FVEEgICAgICJsYWtlaG91c2UiOiB7CiMgTUVUQSAgICAgICAiZGVmYXVsdF9sYWtlaG91c2UiOiAiMjIwMTUwMmMtNzM1YS00ODU1LThiZmQtOGU0NDhmOTEzYzkwIiwKIyBNRVRBICAgICAgICJkZWZhdWx0X2xha2Vob3VzZV9uYW1lIjogImRldl9sYWtlaG91c2UiLAojIE1FVEEgICAgICAgImRlZmF1bHRfbGFrZWhvdXNlX3dvcmtzcGFjZV9pZCI6ICI0YWNhMDUwOC1lOTY2LTQwYTctYTJhMi04NDg2MWIyODFjODMiLAojIE1FVEEgICAgICAgImtub3duX2xha2Vob3VzZXMiOiBbCiMgTUVUQSAgICAgICAgIHsKIyBNRVRBICAgICAgICAgICAiaWQiOiAiMjIwMTUwMmMtNzM1YS00ODU1LThiZmQtOGU0NDhmOTEzYzkwIgojIE1FVEEgICAgICAgICB9CiMgTUVUQSAgICAgICBdCiMgTUVUQSAgICAgfQojIE1FVEEgICB9CiMgTUVUQSB9CgojIENFTEwgKioqKioqKioqKioqKioqKioqKioKCiNsb2FkIHRoZSBOWUMgVGF4aSBkYXRhc2V0IApueWNfZGYgPSBzcGFyay5yZWFkLnBhcnF1ZXQoImFiZnNzOi8vNGFjYTA1MDgtZTk2Ni00MGE3LWEyYTItODQ4NjFiMjgxYzgzQG9uZWxha2UuZGZzLmZhYnJpYy5taWNyb3NvZnQuY29tLzIyMDE1MDJjLTczNWEtNDg1NS04YmZkLThlNDQ4ZjkxM2M5MC9GaWxlcy9OWUMtVGF4aS1HcmVlbi8iKQoKI2Rpc3BsYXkgdG9wIDEwMApkaXNwbGF5KG55Y19kZi5saW1pdCgxMDApKQoKCiMgTUVUQURBVEEgKioqKioqKioqKioqKioqKioqKioKCiMgTUVUQSB7CiMgTUVUQSAgICJsYW5ndWFnZSI6ICJweXRob24iLAojIE1FVEEgICAibGFuZ3VhZ2VfZ3JvdXAiOiAic3luYXBzZV9weXNwYXJrIgojIE1FVEEgfQoKIyBDRUxMICoqKioqKioqKioqKioqKioqKioqCgojIENvdW50IGRpc3RpbmN0IHZlbmRvcklECmRpc3RpbmN0X3ZlbmRvcl9jb3VudCA9IG55Y19kZi5zZWxlY3QoInZlbmRvcklEIikuZGlzdGluY3QoKS5jb3VudCgpCgpwcmludChmIkRpc3RpbmN0IHZlbmRvcklEIGNvdW50OiB7ZGlzdGluY3RfdmVuZG9yX2NvdW50fSIpCgojIE1FVEFEQVRBICoqKioqKioqKioqKioqKioqKioqCgojIE1FVEEgewojIE1FVEEgICAibGFuZ3VhZ2UiOiAicHl0aG9uIiwKIyBNRVRBICAgImxhbmd1YWdlX2dyb3VwIjogInN5bmFwc2VfcHlzcGFyayIKIyBNRVRBIH0KCiMgQ0VMTCAqKioqKioqKioqKioqKioqKioqKgoKIyBDb3VudCB2YWx1ZXMgZm9yIGVhY2ggdmVuZG9ySUQKdmVuZG9yX2NvdW50cyA9IG55Y19kZi5ncm91cEJ5KCJ2ZW5kb3JJRCIpLmNvdW50KCkKdmVuZG9yX2NvdW50cy5zaG93KCkKCiMgTUVUQURBVEEgKioqKioqKioqKioqKioqKioqKioKCiMgTUVUQSB7CiMgTUVUQSAgICJsYW5ndWFnZSI6ICJweXRob24iLAojIE1FVEEgICAibGFuZ3VhZ2VfZ3JvdXAiOiAic3luYXBzZV9weXNwYXJrIgojIE1FVEEgfQoKIyBDRUxMICoqKioqKioqKioqKioqKioqKioqCgpueWNfZGYucHJpbnRTY2hlbWEoKQoKIyBNRVRBREFUQSAqKioqKioqKioqKioqKioqKioqKgoKIyBNRVRBIHsKIyBNRVRBICAgImxhbmd1YWdlIjogInB5dGhvbiIsCiMgTUVUQSAgICJsYW5ndWFnZV9ncm91cCI6ICJzeW5hcHNlX3B5c3BhcmsiCiMgTUVUQSB9Cg==";

var notebookPayload = new 
{
    displayName = notebook_name,
    description = notebook_desc,
    definition = new 
    {        
        parts = new[] 
        {
            new 
            {
                path = "notebook-content.py",
                payload = content_part_b64,
                payloadType = "InlineBase64"
            },
            new 
            {
                path = ".platform",
                payload = platform_part_b64,
                payloadType = "InlineBase64"
            }
        }
    }
};

var uri = $"workspaces/{sample_ws_id}/notebooks";
var jsonPayload = JsonSerializer.Serialize(notebookPayload);

// Call the PostAsync method
string location = await PostAsync(uri, jsonPayload);

// Step 2: Poll the status
Console.WriteLine($"Notebook creation initiated. Location: {location}");

// Step 2: Poll the status
if (!string.IsNullOrEmpty(location))
{
    string finalResponse = await PollStatusAsync(location);
    Console.WriteLine("Final Notebook Response:");
    Console.WriteLine(finalResponse);
}
else
{
    Console.WriteLine("No Location header received; unable to poll.");
}

Token is still valid
Notebook creation initiated. Location: https://wabi-us-east2-d-primary-redirect.analysis.windows.net/v1/operations/9ee625c4-070a-42f2-bad4-20a6ea8f5c29
Token is still valid
Final Notebook Response:
{"status":"Running","createdTimeUtc":"2024-11-21T16:05:45.1046876","lastUpdatedTimeUtc":"2024-11-21T16:05:45.1046876","percentComplete":null,"error":null}


### Update Item (Notebook) with Definition

`POST https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks/{notebookId}/updateDefinition` is the api we need to call. The payload is similar to the creation of a notebook with definition.


In [18]:
// we wil use these:
var notebook_2_update = "ee37a9ae-d1d8-4ac8-b132-debb0279a1b0";
// var notebook_name = "A Sample Notebook4 Updated";
// var notebook_desc = "A sample notebook4 description4 Updated";
notebook_desc = "A sample notebook4 description4 Updated with definition";
// we will also use the same part as the notebook we update is an empty one
var notebookPayload2 = new 
{
    displayName = "A Sample Notebook4 Updated",
    description = notebook_desc,
    definition = new 
    {        
        parts = new[] 
        {
            new 
            {
                path = "notebook-content.py",
                payload = content_part_b64,
                payloadType = "InlineBase64"
            },
            new 
            {
                path = ".platform",
                payload = platform_part_b64,
                payloadType = "InlineBase64"
            }
        }
    }
};

uri = $"workspaces/{sample_ws_id}/notebooks/{notebook_2_update}/updateDefinition";
jsonPayload = JsonSerializer.Serialize(notebookPayload);

// Call the PostAsync method
string location = await PostAsync(uri, jsonPayload);

// Step 2: Poll the status
Console.WriteLine($"Notebook creation initiated. Location: {location}");

// Step 2: Poll the status
if (!string.IsNullOrEmpty(location))
{
    string finalResponse = await PollStatusAsync(location);
    Console.WriteLine("Final Notebook Response:");
    Console.WriteLine(finalResponse);
}
else
{
    Console.WriteLine("No Location header received; unable to poll.");
}


Token is still valid
Notebook creation initiated. Location: https://wabi-us-east2-d-primary-redirect.analysis.windows.net/v1/operations/de0a2f1d-acfa-404d-837f-3469206c478f
Token is still valid
Final Notebook Response:
{"status":"Succeeded","createdTimeUtc":"2024-11-21T16:15:01.3296532","lastUpdatedTimeUtc":"2024-11-21T16:15:02.1265402","percentComplete":100,"error":null}


In [ ]:

using System.Collections.Generic;
using System.Text.Json;
using System.Text.Json.Serialization;

// Define classes matching the JSON structure
public class WorkspaceListResponse
{
    [JsonPropertyName("value")]
    public List<Workspace> Value { get; set; }
}

public class Workspace
{
    [JsonPropertyName("id")]
    public string Id { get; set; }

    // Include other properties if needed
    [JsonPropertyName("name")]
    public string Name { get; set; }
}

public class NotebookListResponse
{
    [JsonPropertyName("value")]
    public List<Notebook> Value { get; set; }
}

public class Notebook
{
    [JsonPropertyName("id")]
    public string Id { get; set; }

    [JsonPropertyName("type")]
    public string Type { get; set; }

    [JsonPropertyName("displayName")]
    public string DisplayName { get; set; }

    [JsonPropertyName("description")]
    public string Description { get; set; }

    [JsonPropertyName("workspaceId")]
    public string WorkspaceId { get; set; }
}

In [ ]:

WorkspaceListResponse workspaceList = JsonSerializer.Deserialize<WorkspaceListResponse>(workspacesResponse);

In [ ]:
// Example: Get notebooks in a workspace
string workspaceId = "4aca0508-e966-40a7-a2a2-84861b281c83";
string notebooksEndpoint = $"workspaces/{workspaceId}/notebooks";
string notebooksResponse = await GetAsync(notebooksEndpoint);
Console.WriteLine("Notebooks in workspace:");
Console.WriteLine(notebooksResponse);


In [ ]:
static string ToBase64(string input)
{
    if (string.IsNullOrEmpty(input))
    {
        throw new ArgumentException("Input string cannot be null or empty", nameof(input));
    }

    // Convert the string to a byte array
    byte[] byteArray = Encoding.UTF8.GetBytes(input);

    // Convert the byte array to a Base64 string
    return Convert.ToBase64String(byteArray);
}

In [ ]:
using System;
using System.IO;
// try to push a notebook between workspaces. 
// /Users/yoavdobrin/workspace/fta/fabric-deployment-hub/notebooks/items/notebooks/Notebook 2.Notebook
string src_location = "./items/notebooks/Notebook 2.Notebook";
// verify there are files in the src_location
if (Directory.Exists(src_location))
    {
        // List all files in the folder
        string[] files = Directory.GetFiles(src_location);
        Console.WriteLine("Files:");
        foreach (string file in files)
        {
            string content = File.ReadAllText(file);
            string base64Content = ToBase64(content);
            string base64FilePath = $"{file}.b64";
            // Write the Base64 content to the new file
            File.WriteAllText(base64FilePath, base64Content);

            Console.WriteLine($"Base64 file created: {base64FilePath}");
            // Console.WriteLine(file);
        }
    }
    else
    {
        Console.WriteLine("Folder does not exist.");
    }

In [ ]:

using System.Text.Json;

// Fetch workspaces
string workspacesResponse = await GetAsync("workspaces");

// Parse workspaces response
WorkspaceListResponse workspaceList = JsonSerializer.Deserialize<WorkspaceListResponse>(workspacesResponse);

// Check if workspaces are found
if (workspaceList?.Value != null && workspaceList.Value.Count > 0)
{
    // Iterate over each workspace
    foreach (var workspace in workspaceList.Value)
    {
        string workspaceId = workspace.Id;
        string notebooksEndpoint = $"workspaces/{workspaceId}/notebooks";
        string notebooksResponse = await GetAsync(notebooksEndpoint);

        // Parse notebooks response
        NotebookListResponse notebookList = JsonSerializer.Deserialize<NotebookListResponse>(notebooksResponse);

        // Output the notebooks
        Console.WriteLine($"Notebooks in Workspace '{workspace.Name}' (ID: {workspaceId}):");
        if (notebookList?.Value != null && notebookList.Value.Count > 0)
        {
            foreach (var notebook in notebookList.Value)
            {
                Console.WriteLine($"- {notebook.DisplayName} (ID: {notebook.Id})");       
                // Fetch notebook definition
                string notebookId = notebook.Id;
                string getDefinitionEndpoint = $"workspaces/{workspaceId}/notebooks/{notebookId}/getDefinition";

                // The getDefinition endpoint may or may not require a body. If it requires an empty JSON object, we can send that.
                HttpContent content = new StringContent("{}", System.Text.Encoding.UTF8, "application/json");

                string definitionResponse = await PostAsync(getDefinitionEndpoint, content);

                // Output or process the definition
                Console.WriteLine($"Definition of Notebook '{notebook.DisplayName}' (ID: {notebookId}):");
                Console.WriteLine(definitionResponse);
                Console.WriteLine(); // Add an empty line for readability
          
            }
        }
        else
        {
            Console.WriteLine("No notebooks found in this workspace.");
        }

        Console.WriteLine(); // Add an empty line for readability
    }
}
else
{
    Console.WriteLine("No workspaces found.");
}

In [ ]:

// Example: Post to get notebook definition
string notebookId = "your_notebook_id";
string getDefinitionEndpoint = $"workspaces/{workspaceId}/notebooks/{notebookId}/getDefinition";
HttpContent content = new StringContent(""); // Replace with actual content if required
string definitionResponse = await PostAsync(getDefinitionEndpoint, content);
Console.WriteLine("Notebook definition:");
Console.WriteLine(definitionResponse);

In [ ]:
// Create client 
HttpClient client = new HttpClient(); 
client.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", result.AccessToken); 
string baseUrl = "https://api.fabric.microsoft.com/v1/"; 
client.BaseAddress = new Uri(baseUrl); 

// Call list workspaces API 
HttpResponseMessage response = await client.GetAsync("workspaces"); 
string responseBody = await response.Content.ReadAsStringAsync(); 
Console.WriteLine(responseBody);

In [ ]:


IConfidentialClientApplication app = ConfidentialClientApplicationBuilder.Create(clientId)
    .WithClientSecret(client_secret)
    .WithAuthority(authority)
    .Build();

try
    {
        AuthenticationResult result = await app.AcquireTokenForClient(scopes)
            .ExecuteAsync();
        Console.WriteLine("Access Token:");
        Console.WriteLine(result.AccessToken);

            // Use the access token to call your API
        HttpClient client = new HttpClient(); 
        client.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", result.AccessToken); 
        string baseUrl = "https://api.fabric.microsoft.com/v1/"; 
        client.BaseAddress = new Uri(baseUrl); 
           // Call list workspaces API 
        HttpResponseMessage response = await client.GetAsync("workspaces"); 
        string responseBody = await response.Content.ReadAsStringAsync(); 
        Console.WriteLine(responseBody);
    }catch (MsalServiceException ex)
    {
        Console.WriteLine($"MsalServiceException: {ex.Message}");
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Exception: {ex.Message}");
    }